# DMML2- Initial Data Exporation and Tensorflow Model Building

## Import relevant modules

In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.optimizers import RMSprop
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam, SGD, Adagrad
from sklearn import metrics



In [35]:
raw = pd.read_csv("data/220306_slugs_dataset.csv")
raw = raw.drop(['loadCase', 'out_supReac_kN_n3', 'out_supReac_kN_n4', 'out_verDisp_mm_n5'], axis=1)

## Feature Engineering

In [36]:
# Create some new features
raw['densDiff'] = raw['slugDen'] - raw['pockDen']
raw['centrFor'] = raw['slugVel']**2 * raw['densDiff']

## EDA

In [37]:
raw.head(5)

,slugDen,slugLen,pockDen,pockLen,slugVel,basePer,baseAmp,basePha,simTime,out_oopDisp_mm_n2,out_oopDisp_mm_n5,out_endBend_kNm_n6,densDiff,centrFor
0,785.67,6.47,234.16,10.06,5.77,12.55,0.85,74.10,45.8,197.45,180.47,198.49,551.51,18361.367279
1,705.03,10.36,190.44,11.25,7.42,11.63,0.80,59.11,44.0,186.85,178.72,238.07,514.59,28331.472876
2,741.98,11.64,159.09,12.52,7.42,11.54,0.65,65.23,48.2,155.47,163.53,180.64,582.89,32091.824996
3,731.88,8.82,167.88,9.84,9.52,11.48,0.67,67.85,30.8,169.47,163.76,233.31,564.00,51115.545600
4,669.21,11.46,213.87,8.56,8.18,12.86,0.81,56.68,37.7,223.64,242.37,219.33,455.34,30467.892216


In [38]:
raw.describe()

,slugDen,slugLen,pockDen,pockLen,slugVel,basePer,baseAmp,basePha,simTime,out_oopDisp_mm_n2,out_oopDisp_mm_n5,out_endBend_kNm_n6,densDiff,centrFor
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,49975.000000,49975.000000,49975.000000,50000.000000,50000.000000
mean,749.872761,10.000962,199.758501,10.010496,7.993627,11.999761,0.699835,60.026022,41.570900,182.077874,187.301203,218.388931,550.114260,37334.569111
std,75.250796,1.988878,49.985999,1.998710,1.990718,1.001198,0.100237,10.025567,15.655315,31.026056,44.422666,59.079226,90.207377,19030.478577
min,439.010000,0.750000,9.470000,2.070000,0.400000,8.050000,0.270000,20.910000,15.200000,73.240000,73.890000,83.330000,182.810000,69.683200
25%,699.077500,8.660000,166.197500,8.650000,6.657500,11.330000,0.630000,53.210000,32.600000,162.600000,160.960000,177.360000,488.990000,23381.323576
50%,749.200000,10.010000,199.690000,10.010000,7.990000,12.000000,0.700000,59.970000,38.500000,179.730000,180.340000,209.190000,549.770000,34584.367189
75%,801.130000,11.340000,233.482500,11.370000,9.340000,12.680000,0.770000,66.820000,46.700000,198.140000,203.755000,247.500000,610.910000,48180.168187
max,1066.900000,18.220000,404.010000,17.860000,16.810000,16.380000,1.110000,106.850000,884.800000,497.920000,711.910000,700.150000,951.300000,169747.298550


In [39]:
rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = raw.corr()
corr.style.background_gradient(cmap='RdBu_r')


,slugDen,slugLen,pockDen,pockLen,slugVel,basePer,baseAmp,basePha,simTime,out_oopDisp_mm_n2,out_oopDisp_mm_n5,out_endBend_kNm_n6,densDiff,centrFor
slugDen,1.000000,0.002973,0.003178,-0.000839,0.002337,-0.007276,0.004203,0.000084,-0.005237,0.107886,0.136626,0.196760,0.832437,0.268792
slugLen,0.002973,1.000000,0.003706,-0.000624,0.011451,0.002834,0.000198,0.005640,0.200271,-0.098746,-0.079360,0.035261,0.000427,0.011067
pockDen,0.003178,0.003706,1.000000,0.006535,-0.000133,0.004100,-0.001499,0.000611,0.002585,-0.055460,-0.072310,-0.076405,-0.551472,-0.177761
pockLen,-0.000839,-0.000624,0.006535,1.000000,-0.000667,-0.000209,0.000465,-0.002671,0.209454,-0.094856,-0.084068,-0.036801,-0.004321,-0.001595
slugVel,0.002337,0.011451,-0.000133,-0.000667,1.000000,-0.001702,0.001361,-0.004626,-0.775522,0.348185,0.510519,0.713178,0.002024,0.920490
basePer,-0.007276,0.002834,0.004100,-0.000209,-0.001702,1.000000,0.000866,0.007024,0.002168,-0.003866,0.004313,0.011802,-0.008341,-0.004311
baseAmp,0.004203,0.000198,-0.001499,0.000465,0.001361,0.000866,1.000000,0.001621,0.001092,0.689821,0.402204,0.334415,0.004336,0.004044
basePha,0.000084,0.005640,0.000611,-0.002671,-0.004626,0.007024,0.001621,1.000000,0.007794,-0.008840,-0.030930,-0.053166,-0.000269,-0.002115
simTime,-0.005237,0.200271,0.002585,0.209454,-0.775522,0.002168,0.001092,0.007794,1.000000,-0.273195,-0.378817,-0.483507,-0.005801,-0.652962
out_oopDisp_mm_n2,0.107886,-0.098746,-0.055460,-0.094856,0.348185,-0.003866,0.689821,-0.008840,-0.273195,1.000000,0.913198,0.660997,0.120739,0.384655


## Data Cleaning

In [40]:

raw = raw.dropna()

scaler = MinMaxScaler()
refined_df = pd.DataFrame(scaler.fit_transform(raw), columns = raw.columns, index = raw.index)
print(refined_df)

        slugDen   slugLen   pockDen   pockLen   slugVel   basePer   baseAmp  \
0      0.552103  0.327418  0.569499  0.506016  0.327239  0.540216  0.690476   
1      0.423673  0.550086  0.458686  0.581381  0.427788  0.429772  0.630952   
2      0.482521  0.623354  0.379226  0.661811  0.427788  0.418968  0.452381   
3      0.466435  0.461935  0.401506  0.492084  0.555759  0.411765  0.476190   
4      0.366625  0.613051  0.518072  0.411020  0.474101  0.577431  0.642857   
...         ...       ...       ...       ...       ...       ...       ...   
49995  0.467136  0.475673  0.413672  0.499050  0.517367  0.567827  0.416667   
49996  0.564112  0.610189  0.455163  0.517416  0.411335  0.349340  0.464286   
49997  0.289079  0.621065  0.415294  0.462318  0.221207  0.499400  0.535714   
49998  0.422287  0.424728  0.518401  0.622546  0.669714  0.228091  0.440476   
49999  0.527592  0.551231  0.459446  0.687144  0.659354  0.456182  0.559524   

        basePha   simTime  out_oopDisp_mm_n2  out_o

## Test Train

In [41]:
# Create train and test splits (20% test)
train_data, test_data = train_test_split(refined_df, test_size=0.20, random_state=0) 

x_train = train_data.drop(["out_oopDisp_mm_n2", "out_oopDisp_mm_n5", "out_endBend_kNm_n6"], axis=1)
y_train = train_data[["out_oopDisp_mm_n2", "out_oopDisp_mm_n5", "out_endBend_kNm_n6"]]
x_test =  test_data.drop(["out_oopDisp_mm_n2", "out_oopDisp_mm_n5", "out_endBend_kNm_n6"], axis=1)
y_test =  test_data[["out_oopDisp_mm_n2", "out_oopDisp_mm_n5", "out_endBend_kNm_n6"]]

## Modelling

In [42]:
def get_model(input_dimensions, output_dimensions):
    
    model = Sequential()
    model.add(Dense(128, input_dim=input_dimensions, activation='relu'))
    model.add(Dropout(.1))
    for i in range(1,10):
        model.add(Dense(64, activation='relu'))
    
    

    
    model.add(Dense(output_dimensions, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=Adam(), metrics=['mean_squared_error'])
    return model

In [43]:
model = get_model(11, 3)
model.fit(x_train, y_train, epochs=10, batch_size=10)


Epoch 1/10
3998/3998 [==============================] - 9s 2ms/step - loss: 0.0026 - mean_squared_error: 0.0026
Epoch 2/10
3998/3998 [==============================] - 8s 2ms/step - loss: 0.0020 - mean_squared_error: 0.0020
Epoch 3/10
3998/3998 [==============================] - 8s 2ms/step - loss: 0.0017 - mean_squared_error: 0.0017
Epoch 4/10
3998/3998 [==============================] - 8s 2ms/step - loss: 0.0015 - mean_squared_error: 0.0015
Epoch 5/10
3998/3998 [==============================] - 8s 2ms/step - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 6/10
3998/3998 [==============================] - 8s 2ms/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 7/10
3998/3998 [==============================] - 8s 2ms/step - loss: 0.0012 - mean_squared_error: 0.0012
Epoch 8/10
3998/3998 [==============================] - 8s 2ms/step - loss: 0.0012 - mean_squared_error: 0.0012
Epoch 9/10
3998/3998 [==============================] - 8s 2ms/step - loss: 0.0011 - mean_squared_error:

## Evaluation

In [44]:
preds = model.predict(x_test)
r2 = metrics.r2_score(y_test, preds)
print(r2)

0.8203295135444039


## Optimzation

.....